In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [4]:
df['model'].value_counts()

F-150 XLT             780
M3 Base               682
X5 xDrive35i          615
F-150 Lariat          496
Mustang GT Premium    466
                     ... 
Suburban 2500           1
TLX Tech                1
Rogue Sport S           1
A8 4.0                  1
S-10 LS                 1
Name: model, Length: 1827, dtype: int64

In [5]:
df.describe()

,id,model_year,milage,price
count,54273.000000,54273.000000,54273.000000,5.427300e+04
mean,27136.000000,2015.091979,72746.175667,3.921844e+04
std,15667.409917,5.588909,50469.490448,7.282634e+04
min,0.000000,1974.000000,100.000000,2.000000e+03
25%,13568.000000,2012.000000,32268.000000,1.550000e+04
50%,27136.000000,2016.000000,66107.000000,2.800000e+04
75%,40704.000000,2019.000000,102000.000000,4.500000e+04
max,54272.000000,2024.000000,405000.000000,2.954083e+06


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            54273 non-null  int64 
 1   brand         54273 non-null  object
 2   model         54273 non-null  object
 3   model_year    54273 non-null  int64 
 4   milage        54273 non-null  int64 
 5   fuel_type     54273 non-null  object
 6   engine        54273 non-null  object
 7   transmission  54273 non-null  object
 8   ext_col       54273 non-null  object
 9   int_col       54273 non-null  object
 10  accident      54273 non-null  object
 11  clean_title   54273 non-null  object
 12  price         54273 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 5.4+ MB


In [7]:
df['horsepower']=df['engine'].apply(lambda x:float(x[:4]) if x[5:7]=='HP' else 331.687)

In [8]:
df['cylinder_config']=df['engine'].apply(lambda engine_str: "V" if "V" in engine_str else "Straight" if "Straight" in engine_str else "unknownconfig")

In [9]:
df['cylinder_config']

0                    V
1             Straight
2        unknownconfig
3             Straight
4                    V
             ...      
54268    unknownconfig
54269    unknownconfig
54270                V
54271                V
54272    unknownconfig
Name: cylinder_config, Length: 54273, dtype: object

In [10]:
cylinder_config=pd.get_dummies(df['cylinder_config'])
df=pd.concat([df,cylinder_config],axis=1)

In [11]:
import re
v1=df['engine'].str.extract(r'\s(\d+)\s')

In [12]:
v1=pd.DataFrame({'index':range(0,54273),'number':v1[0]})
# v1[0]

In [13]:
v2=df['engine'].str.extract(r'\s\w(\d+)\s')

In [14]:
v2=pd.DataFrame({'index':range(0,54273),'number':v2[0]})

In [15]:
newv=pd.merge(v1, v2, on='index', how='outer')

In [16]:
newv.head()

,index,number_x,number_y
0,0,NaN,6
1,1,6,NaN
2,2,8,NaN
3,3,6,NaN
4,4,NaN,6


In [17]:
df['v']=newv['number_x'].combine_first(newv['number_y'])

In [18]:
df['v']=df['v'].fillna('6')
df['v'] = df['v'].astype(int)

In [19]:
l1=df['engine'].str.extract(r'\s(\d+\.\d+)\w\s')
l1=pd.DataFrame({'index':range(0,54273),'number':l1[0]})

In [20]:
l2=df['engine'].str.extract(r'\b(\d+\.\d+)\sLiter\b')
l2=pd.DataFrame({'index':range(0,54273),'number':l2[0]})

In [21]:
newv=pd.merge(l1, l2, on='index', how='outer')
df['L']=newv['number_x'].combine_first(newv['number_y'])


In [22]:
df['L']=df['L'].fillna('3.72')
df['L'] = df['L'].astype(float)

In [23]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horsepower,cylinder_config,Straight,V,unknownconfig,v,L
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000,375.0,V,0,1,0,6,3.5
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250,300.0,Straight,1,0,0,6,3.0
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000,300.0,unknownconfig,0,0,1,8,4.2
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500,335.0,Straight,1,0,0,6,3.0
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850,200.0,V,0,1,0,6,3.8


In [24]:
df['transmission'].value_counts()

A/T                                                   16757
8-Speed A/T                                            7287
Transmission w/Dual Shift Mode                         6454
6-Speed A/T                                            5944
6-Speed M/T                                            3618
7-Speed A/T                                            3415
10-Speed A/T                                           2187
8-Speed Automatic                                      1189
5-Speed A/T                                            1185
9-Speed A/T                                            1073
4-Speed A/T                                             764
5-Speed M/T                                             718
CVT Transmission                                        692
10-Speed Automatic                                      642
6-Speed Automatic                                       516
Automatic                                               441
M/T                                     

In [25]:
# df['tranmission_no']=df['transmission'].str.extract(r'(\d+)')
def extract_transmission_features(transmission_str):
    if 'A/T' in transmission_str or 'Automatic' in transmission_str or 'Auto' in transmission_str or 'automatic' in transmission_str or 'auto' in transmission_str:
        transmission_type = 'A/T'
        num_gears = re.search(r'(\d+)-Speed', transmission_str)
        num_gears = int(num_gears.group(1)) if num_gears else None
        transmission_features = transmission_str.replace(str(num_gears) + '-Speed A/T', '').strip()
    elif 'M/T' in transmission_str or 'Manual' in transmission_str or 'manual'in transmission_str:
        transmission_type = 'M/T'
        num_gears = re.search(r'(\d+)-Speed', transmission_str)
        num_gears = int(num_gears.group(1)) if num_gears else None
        transmission_features = transmission_str.replace(str(num_gears) + '-Speed M/T', '').strip()
    elif 'Dual' in transmission_str or 'dual' in transmission_str:
        transmission_type = 'Dual'
        num_gears = 0
        transmission_features = transmission_str.strip()
    else:
        transmission_type = 'Unknown'
        num_gears = 0
        transmission_features = 'Unknown'
    return transmission_type, num_gears,transmission_features

In [26]:
df[['transmission_type', 'num_gears','transmission_features']]=df['transmission'].apply(extract_transmission_features).apply(pd.Series)

In [27]:
df['num_gears']=df['num_gears'].fillna(1)
df['num_gears']=df['num_gears'].astype(int)

In [28]:
df['transmission_features']=df['transmission_features'].replace({"":"idontknow"})
df['transmission_features']=df['transmission_features'].fillna('idontknow')

In [29]:
arr=df['transmission_features'].apply(lambda x:sorted([i+"_trans" for i in re.findall(r'\b[a-zA-Z]+\b',x)]))
colset=[]
for i in range(54273):
    for j in arr[i]:
        colset.append(j)
transdata=list(set(colset))
print(transdata)

['Shift_trans', 'T_trans', 'CVT_trans', 'Auto_trans', 'Dual_trans', 'O_trans', 'Manual_trans', 'w_trans', 'idontknow_trans', 'Unknown_trans', 'Automatic_trans', 'Speed_trans', 'DCT_trans', 'A_trans', 'Overdrive_trans', 'Electronically_trans', 'Transmission_trans', 'Controlled_trans', 'Mode_trans', 'M_trans', 'with_trans']


In [30]:
for i in transdata:
    df[i]=0
for i in range(54273):
    for j in arr[i]:
        if j in transdata:
            df.loc[i,j]=1

In [31]:
acci=pd.get_dummies(df['accident'])
df=pd.concat([df,acci],axis=1)

In [32]:
acci=pd.get_dummies(df['transmission_type'])
df=pd.concat([df,acci],axis=1)

In [33]:
# int_col=pd.get_dummies(df['int_col'])
# df=pd.concat([df,int_col],axis=1)
# df['int_col']=df['int_col'].astype('category').cat.codes
# encoder1 = LabelEncoder()
# encoder1.fit(df['int_col'])

# # Transform training data
# df['int_col'] = encoder1.transform(df['int_col'])
intdict={}
for i in range(len(list(set(df['int_col'])))):
    intdict[list(set(df['int_col']))[i]]=i
print(intdict)
df['int_col']=df['int_col'].replace(intdict)

{'Charles Blue': 0, 'Titan Black / Quarzit': 1, 'Pimento Red w/Ebony': 2, 'Camel': 3, 'Hotspur Hide': 4, 'Oyster W/Contrast': 5, 'Blue': 6, 'Shara Beige': 7, 'Medium Earth Gray': 8, 'Nougat Brown': 9, 'Mocha': 10, 'Espresso': 11, 'Cocoa / Dune': 12, 'Black / Brown': 13, 'Dark Auburn': 14, 'Orange': 15, 'Nero Ade': 16, 'Rock Gray': 17, 'Very Light Cashmere': 18, 'Dark Gray': 19, 'Blk': 20, 'Ceramic': 21, 'Almond Beige': 22, 'Beige': 23, 'Red/Black': 24, 'Parchment.': 25, 'Macchiato': 26, 'Canberra Beige': 27, 'Mistral Gray / Raven': 28, 'Black / Stone Grey': 29, 'Black': 30, 'Portland': 31, 'White / Brown': 32, 'Cappuccino': 33, 'Kyalami Orange': 34, 'Graystone': 35, 'Mesa': 36, 'Charcoal Black': 37, 'Sport': 38, 'Global Black': 39, 'Ebony Black': 40, 'Giallo Taurus / Nero Ade': 41, 'BLACK': 42, 'Red': 43, 'Black Onyx': 44, 'Dark Ash': 45, 'Mountain Brown': 46, 'Sahara Tan': 47, 'Cloud': 48, 'Beluga Hide': 49, 'Deep Chestnut': 50, 'Cobalt Blue': 51, 'Amber': 52, 'Deep Garnet': 53, 'Oyst

In [34]:
clean_title=pd.get_dummies(df['clean_title'])
df=pd.concat([df,clean_title],axis=1)

In [35]:
# ext_col=pd.get_dummies(df['ext_col'])
# df=pd.concat([df,ext_col],axis=1)
# df['int_col']=df['int_col'].astype('category').cat.codes
# encoder2 = LabelEncoder()
# encoder2.fit(df['ext_col'])

# # Transform training data
# df['ext_col'] = encoder2.transform(df['ext_col'])
extdict={}
for i in range(len(list(set(df['ext_col'])))):
    extdict[list(set(df['ext_col']))[i]]=i
print(intdict)
df['ext_col']=df['ext_col'].replace(extdict)
df['ext_col']

{'Charles Blue': 0, 'Titan Black / Quarzit': 1, 'Pimento Red w/Ebony': 2, 'Camel': 3, 'Hotspur Hide': 4, 'Oyster W/Contrast': 5, 'Blue': 6, 'Shara Beige': 7, 'Medium Earth Gray': 8, 'Nougat Brown': 9, 'Mocha': 10, 'Espresso': 11, 'Cocoa / Dune': 12, 'Black / Brown': 13, 'Dark Auburn': 14, 'Orange': 15, 'Nero Ade': 16, 'Rock Gray': 17, 'Very Light Cashmere': 18, 'Dark Gray': 19, 'Blk': 20, 'Ceramic': 21, 'Almond Beige': 22, 'Beige': 23, 'Red/Black': 24, 'Parchment.': 25, 'Macchiato': 26, 'Canberra Beige': 27, 'Mistral Gray / Raven': 28, 'Black / Stone Grey': 29, 'Black': 30, 'Portland': 31, 'White / Brown': 32, 'Cappuccino': 33, 'Kyalami Orange': 34, 'Graystone': 35, 'Mesa': 36, 'Charcoal Black': 37, 'Sport': 38, 'Global Black': 39, 'Ebony Black': 40, 'Giallo Taurus / Nero Ade': 41, 'BLACK': 42, 'Red': 43, 'Black Onyx': 44, 'Dark Ash': 45, 'Mountain Brown': 46, 'Sahara Tan': 47, 'Cloud': 48, 'Beluga Hide': 49, 'Deep Chestnut': 50, 'Cobalt Blue': 51, 'Amber': 52, 'Deep Garnet': 53, 'Oyst

0         13
1         92
2         85
3        189
4        206
        ... 
54268    206
54269     92
54270    206
54271     74
54272     92
Name: ext_col, Length: 54273, dtype: int64

In [36]:
fuel_type=pd.get_dummies(df['fuel_type'])
df=pd.concat([df,fuel_type],axis=1)

In [37]:
brand=pd.get_dummies(df['brand'])
df=pd.concat([df,brand],axis=1)

In [38]:
# df['model']=df['model'].astype('category').cat.codes
def extract_model_features(model):
    """
    This function extracts features from the model column creating new features 
    and enriching the dataset.
    """
    parts = model.split()
    base_model = parts[0] if parts else model
    model_variant = parts[1] if len(parts) > 1 else ''
    additional_features = ''.join(parts[2:]) if len(parts) > 2 else ''
    return base_model, model_variant, additional_features
df[['base_model', 'model_variant', 'additional_features']] =df['model'].apply(lambda x: pd.Series(extract_model_features(x)))

In [39]:
base_modeldict={}
for i in range(len(list(set(df['base_model'])))):
    base_modeldict[list(set(df['base_model']))[i]]=i
df['base_model']=df['base_model'].replace(base_modeldict)

model_variantdict={}
for i in range(len(list(set(df['model_variant'])))):
    model_variantdict[list(set(df['model_variant']))[i]]=i
df['model_variant']=df['model_variant'].replace(model_variantdict)

additional_featuresdict={}
for i in range(len(list(set(df['additional_features'])))):
    additional_featuresdict[list(set(df['additional_features']))[i]]=i
df['additional_features']=df['additional_features'].replace(additional_featuresdict)


# df['model_variant']=df['model_variant'].astype('category').cat.codes
# df['additional_features']=df['additional_features'].astype('category').cat.codes

In [40]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,...,Scion,Subaru,Suzuki,Tesla,Toyota,Volkswagen,Volvo,base_model,model_variant,additional_features
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,13,93,...,0,0,0,0,0,0,0,127,379,0
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,92,30,...,0,0,0,0,0,0,0,359,68,0
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,85,23,...,0,0,0,0,0,0,0,324,72,0
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,189,105,...,0,0,0,0,0,0,0,167,126,0
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,206,30,...,0,0,0,0,0,0,0,255,373,0


In [41]:
df.drop(columns=['brand','fuel_type','accident','clean_title','id','model'])

,model_year,milage,engine,transmission,ext_col,int_col,price,horsepower,cylinder_config,Straight,...,Scion,Subaru,Suzuki,Tesla,Toyota,Volkswagen,Volvo,base_model,model_variant,additional_features
0,2018,74349,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,13,93,11000,375.000,V,0,...,0,0,0,0,0,0,0,127,379,0
1,2007,80000,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,92,30,8250,300.000,Straight,1,...,0,0,0,0,0,0,0,359,68,0
2,2009,91491,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,85,23,15000,300.000,unknownconfig,0,...,0,0,0,0,0,0,0,324,72,0
3,2022,2437,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,189,105,63500,335.000,Straight,1,...,0,0,0,0,0,0,0,167,126,0
4,2001,111000,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,206,30,7850,200.000,V,0,...,0,0,0,0,0,0,0,255,373,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54268,2017,29000,445.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,206,105,29000,445.000,unknownconfig,0,...,0,0,0,0,0,0,0,220,237,0
54269,2015,94634,220.0HP 2.0L 4 Cylinder Engine Flex Fuel Capab...,6-Speed A/T,92,30,6500,220.000,unknownconfig,0,...,0,0,0,0,0,0,0,275,453,147
54270,2013,40989,420.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,206,30,18950,420.000,V,0,...,0,0,0,0,0,0,0,280,445,0
54271,2023,1518,4.0L H6 24V GDI DOHC,8-Speed Automatic with Auto-Shift,74,105,194965,331.687,V,0,...,0,0,0,0,0,0,0,79,116,422


In [42]:
arr=df['engine'].apply(lambda x:sorted(re.findall(r'\b[a-zA-Z]+\b',x)))
# arr[0]
colset=[]
for i in range(54273):
    for j in arr[i]:
        colset.append(j)

In [43]:
enginedata=['AH',
 'Battery',
 'Capability',
 'Cylinder',
 'DDI',
 'DOHC',
 'Diesel',
 'Electric',
 'Engine',
 'Flat',
 'Flex',
 'Flexible',
 'Fuel',
 'GDI',
 'GTDI',
 'Gas',
 'Gasoline',
 'Hybrid',
 'Hydrogen',
 'In',
 'Intercooled',
 'Liter',
 'MPFI',
 'Mild',
 'Motor',
 'OHV',
 'PDI',
 'Plug',
 'Range',
 'Rotary',
 'SC',
 'SOHC',
 'Standard',
 'Supercharged',
 'System',
 'TFSI',
 'TSI',
 'Turbo',
 'Twin',
 'ULEV',
 'VTEC',
 'engine']

In [44]:
for i in enginedata:
    df[i]=0
for i in range(54273):
    for j in arr[i]:
        if j in enginedata:
            df.loc[i,j]=1

In [45]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,...,SOHC,Standard,Supercharged,System,TFSI,TSI,Turbo,Twin,ULEV,VTEC
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,0,10-Speed A/T,13,93,...,0,0,0,0,0,0,0,0,0,0
1,1,BMW,335 i,2007,80000,Gasoline,0,6-Speed M/T,92,30,...,0,0,0,0,0,0,0,0,0,0
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,0,6-Speed A/T,85,23,...,0,0,0,0,0,0,0,0,0,0
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,0,Transmission w/Dual Shift Mode,189,105,...,0,0,0,0,0,0,0,0,0,0
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,0,A/T,206,30,...,0,0,0,0,0,0,0,0,0,0


In [46]:
df=df.dropna(axis=1)

In [47]:
df=df.drop(columns=['brand','fuel_type','ext_col','int_col','accident','clean_title','id','model','transmission_type','transmission_features','cylinder_config'],axis=1)

In [48]:
df.head()

,model_year,milage,engine,transmission,price,horsepower,Straight,V,unknownconfig,v,...,SOHC,Standard,Supercharged,System,TFSI,TSI,Turbo,Twin,ULEV,VTEC
0,2018,74349,0,10-Speed A/T,11000,375.0,0,1,0,6,...,0,0,0,0,0,0,0,0,0,0
1,2007,80000,0,6-Speed M/T,8250,300.0,1,0,0,6,...,0,0,0,0,0,0,0,0,0,0
2,2009,91491,0,6-Speed A/T,15000,300.0,0,0,1,8,...,0,0,0,0,0,0,0,0,0,0
3,2022,2437,0,Transmission w/Dual Shift Mode,63500,335.0,1,0,0,6,...,0,0,0,0,0,0,0,0,0,0
4,2001,111000,0,A/T,7850,200.0,0,1,0,6,...,0,0,0,0,0,0,0,0,0,0


In [49]:
target=df['price']
df=df.drop(columns=['transmission'])

In [50]:
# df['noyear']=pd.cut(df['model_year'],9,labels=[str(i) for i in range(9)])

In [51]:
df=df.drop(columns=['price',])

In [52]:
df.columns.to_list()

['model_year',
 'milage',
 'engine',
 'horsepower',
 'Straight',
 'V',
 'unknownconfig',
 'v',
 'L',
 'num_gears',
 'Shift_trans',
 'T_trans',
 'CVT_trans',
 'Auto_trans',
 'Dual_trans',
 'O_trans',
 'Manual_trans',
 'w_trans',
 'idontknow_trans',
 'Unknown_trans',
 'Automatic_trans',
 'Speed_trans',
 'DCT_trans',
 'A_trans',
 'Overdrive_trans',
 'Electronically_trans',
 'Transmission_trans',
 'Controlled_trans',
 'Mode_trans',
 'M_trans',
 'with_trans',
 'At least 1 accident or damage reported',
 'None reported',
 'A/T',
 'Dual',
 'M/T',
 'Unknown',
 'Yes',
 'Diesel',
 'E85 Flex Fuel',
 'Gasoline',
 'Hybrid',
 'Plug-In Hybrid',
 'not supported',
 '–',
 'Acura',
 'Alfa',
 'Aston',
 'Audi',
 'BMW',
 'Bentley',
 'Bugatti',
 'Buick',
 'Cadillac',
 'Chevrolet',
 'Chrysler',
 'Dodge',
 'FIAT',
 'Ferrari',
 'Ford',
 'GMC',
 'Genesis',
 'Honda',
 'Hummer',
 'Hyundai',
 'INFINITI',
 'Jaguar',
 'Jeep',
 'Kia',
 'Lamborghini',
 'Land',
 'Lexus',
 'Lincoln',
 'Lotus',
 'Lucid',
 'MINI',
 'Maserat

In [53]:
!pip3 install catboost

In [54]:
# df['noyear']=df['noyear'].astype(int)

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()
stanscaler=StandardScaler()
# dft=stanscaler.fit_transform(df)
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.00000001, random_state=42)

In [56]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
catboost=CatBoostRegressor(loss_function='RMSE',depth=5,random_seed=67,iterations=1500,learning_rate=0.01,verbose=False)
# catboost=RandomForestRegressor( n_estimators=50, random_state=42)
catboost.fit(X_train, y_train)

In [57]:
from sklearn.metrics import mean_squared_error
y_pred = catboost.predict(X_test)
rmse1 = np.sqrt(mean_squared_error(y_test, y_pred))
print("The Root mean square is " + str(rmse1))

The Root mean square is 17550.547766928095


In [58]:
import random
df1=pd.read_csv('test.csv')
df1.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,54273,Mercedes-Benz,E-Class E 350,2014,73000,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes
1,54274,Lexus,RX 350 Base,2015,128032,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Black,None reported,Yes
2,54275,Mercedes-Benz,C-Class C 300,2015,51983,Gasoline,241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Blue,White,None reported,Yes
3,54276,Land,Rover Range Rover 5.0L Supercharged Autobiogra...,2018,29500,Gasoline,518.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,White,At least 1 accident or damage reported,Yes
4,54277,BMW,X6 xDrive40i,2020,90000,Gasoline,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes


In [59]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            36183 non-null  int64 
 1   brand         36183 non-null  object
 2   model         36183 non-null  object
 3   model_year    36183 non-null  int64 
 4   milage        36183 non-null  int64 
 5   fuel_type     36183 non-null  object
 6   engine        36183 non-null  object
 7   transmission  36183 non-null  object
 8   ext_col       36183 non-null  object
 9   int_col       36183 non-null  object
 10  accident      36183 non-null  object
 11  clean_title   36183 non-null  object
dtypes: int64(3), object(9)
memory usage: 3.3+ MB


In [60]:
df1.replace('-','idontknow',inplace=True)

In [61]:
df1['horsepower']=df1['engine'].apply(lambda x:float(x[:4]) if x[5:7]=='HP' else 306.838)

In [62]:
df1['cylinder_config']=df1['engine'].apply(lambda engine_str: "V" if "V" in engine_str else "Straight" if "Straight" in engine_str else "unknownconfig")
cylinder_config=pd.get_dummies(df1['cylinder_config'])
df1=pd.concat([df1,cylinder_config],axis=1)

In [63]:
v1=df1['engine'].str.extract(r'\s(\d+)\s')
v1=pd.DataFrame({'index':range(0,36183),'number':v1[0]})
v2=df1['engine'].str.extract(r'\s\w(\d+)\s')
v2=pd.DataFrame({'index':range(0,36183),'number':v2[0]})
newv=pd.merge(v1, v2, on='index', how='outer')
newv.head()
df1['v']=newv['number_x'].combine_first(newv['number_y'])
df1['v']=df1['v'].fillna('6')
df1['v'] = df1['v'].astype(int)
l1=df1['engine'].str.extract(r'\s(\d+\.\d+)\w\s')
l1=pd.DataFrame({'index':range(0,36183),'number':l1[0]})
l2=df1['engine'].str.extract(r'\b(\d+\.\d+)\sLiter\b')
l2=pd.DataFrame({'index':range(0,36183),'number':l2[0]})
newv=pd.merge(l1, l2, on='index', how='outer')
df1['L']=newv['number_x'].combine_first(newv['number_y'])
df1['L']=df1['L'].fillna('3.72')
df1['L'] = df1['L'].astype(float)

In [64]:
# df1['tranmission_no']=df1['transmission'].str.extract(r'(\d+)')[0]
df1[['transmission_type', 'num_gears','transmission_features']]=df1['transmission'].apply(extract_transmission_features).apply(pd.Series)

In [65]:
df1[['transmission_type', 'num_gears','transmission_features']]

,transmission_type,num_gears,transmission_features
0,A/T,NaN,A/T
1,A/T,8.0,
2,A/T,7.0,
3,Dual,0.0,Transmission w/Dual Shift Mode
4,A/T,8.0,
...,...,...,...
36178,A/T,10.0,
36179,A/T,4.0,
36180,M/T,5.0,
36181,A/T,10.0,


In [66]:
df1['num_gears']=df1['num_gears'].fillna(1)
df1['num_gears']=df1['num_gears'].astype(int)
df1['transmission_features']=df1['transmission_features'].replace({"":"idontknow"})
df1['transmission_features']=df1['transmission_features'].fillna("idontknow")
acci=pd.get_dummies(df1['transmission_type'])
df1=pd.concat([df1,acci],axis=1)

In [67]:
df1[['transmission_type', 'num_gears','transmission_features']]

,transmission_type,num_gears,transmission_features
0,A/T,1,A/T
1,A/T,8,idontknow
2,A/T,7,idontknow
3,Dual,0,Transmission w/Dual Shift Mode
4,A/T,8,idontknow
...,...,...,...
36178,A/T,10,idontknow
36179,A/T,4,idontknow
36180,M/T,5,idontknow
36181,A/T,10,idontknow


In [68]:
arr=df1['transmission_features'].apply(lambda x:sorted([i+"_trans" for i in re.findall(r'\b[a-zA-Z]+\b',x)]))
colset=[]
for i in range(36183):
    for j in arr[i]:
        colset.append(j)
transdata=list(set(colset))
print(transdata)

['Shift_trans', 'T_trans', 'CVT_trans', 'Auto_trans', 'Dual_trans', 'O_trans', 'Manual_trans', 'w_trans', 'idontknow_trans', 'Speed_trans', 'Automatic_trans', 'Unknown_trans', 'DCT_trans', 'A_trans', 'Overdrive_trans', 'Electronically_trans', 'Transmission_trans', 'Controlled_trans', 'Mode_trans', 'M_trans', 'with_trans']


In [69]:
for i in transdata:
    df1[i]=0
for i in range(36183):
    for j in arr[i]:
        if j in transdata:
            df1.loc[i,j]=1

In [70]:
brand=pd.get_dummies(df1['brand'])
df1=pd.concat([df1,brand],axis=1)
fuel_type=pd.get_dummies(df1['fuel_type'])
df1=pd.concat([df1,fuel_type],axis=1)
# ext_col=pd.get_dummies(df1['ext_col'])
# df1=pd.concat([df1,ext_col],axis=1)
clean_title=pd.get_dummies(df1['clean_title'])
d1f=pd.concat([df1,clean_title],axis=1)
# int_col=pd.get_dummies(df1['int_col'])
# df1=pd.concat([df1,int_col],axis=1)
df1['int_col']=df1['int_col'].apply(lambda x:intdict[x] if x in intdict else random.randint(502, 1000))
df1['ext_col']=df1['ext_col'].apply(lambda x:extdict[x] if x in extdict else random.randint(502, 1000))
acci1=pd.get_dummies(df1['accident'])
df1=pd.concat([df1,acci1],axis=1)

In [71]:
# df1['model']=df1['model'].astype('category').cat.codes
df1[['base_model', 'model_variant', 'additional_features']] =df1['model'].apply(lambda x: pd.Series(extract_model_features(x)))
df1['base_model']=df1['base_model'].apply(lambda x:base_modeldict[x] if x in base_modeldict else random.randint(502, 1000))
df1['model_variant']=df1['model_variant'].apply(lambda x:model_variantdict[x] if x in model_variantdict else random.randint(502, 1000))
df1['additional_features']=df1['additional_features'].apply(lambda x:additional_featuresdict[x] if x in additional_featuresdict else random.randint(502, 1000))

In [72]:
# df1['noyear']=pd.cut(df1['model_year'],9,labels=[str(i) for i in range(9)])
# df1['noyear']=df1['noyear'].astype(int)
# df1['d']=df1['milage']*df1['v']
df1[['base_model', 'model_variant', 'additional_features']]

,base_model,model_variant,additional_features
0,434,117,309
1,90,272,430
2,82,93,502
3,442,289,490
4,220,126,0
...,...,...,...
36178,367,275,0
36179,19,2,157
36180,77,46,0
36181,69,192,0


In [73]:
df1=df1.drop(columns=['brand','fuel_type','ext_col','int_col','accident','clean_title','id','transmission','model','transmission_type','transmission_features','cylinder_config'],axis=1)

In [74]:
arr=df1['engine'].apply(lambda x:sorted(re.findall(r'\b[a-zA-Z]+\b',x)))
# arr[0]
colset=[]
for i in range(36183):
    for j in arr[i]:
        colset.append(j)

In [75]:
enginedata=['AH',
 'Battery',
 'Capability',
 'Cylinder',
 'DDI',
 'DOHC',
 'Diesel',
 'Electric',
 'Engine',
 'Flat',
 'Flex',
 'Flexible',
 'Fuel',
 'GDI',
 'GTDI',
 'Gas',
 'Gasoline',
 'Hybrid',
 'Hydrogen',
 'In',
 'Intercooled',
 'Liter',
 'MPFI',
 'Mild',
 'Motor',
 'OHV',
 'PDI',
 'Plug',
 'Range',
 'Rotary',
 'SC',
 'SOHC',
 'Standard',
 'Supercharged',
 'System',
 'TFSI',
 'TSI',
 'Turbo',
 'Twin',
 'ULEV',
 'VTEC',
 'engine']

In [76]:
for i in enginedata:
    df1[i]=0
for i in range(36183):
    for j in arr[i]:
        if j in enginedata:
            df1.loc[i,j]=1

In [77]:
df1['Cylinder'][1]

1

In [78]:
df1=df1.dropna(axis=1)

In [79]:
# catboost.predict(df1)
# df1.shape
df1=df1.drop(['Saab', 'smart'],axis=1)

In [80]:
df1['Yes']=0
df1['Maybach']=0
df1['Plymouth']=0

In [81]:
# df1=df1.drop(['d'],axis=1)
set(df.columns.tolist())-set(df1.columns.tolist())
set(df1.columns.tolist())-set(df.columns.tolist())

set()

In [82]:
result=pd.DataFrame({'id':pd.read_csv('test.csv')['id'],'price':catboost.predict(df1)})

In [83]:
result.to_csv('out.csv',index=False)

In [84]:
catboost.predict(df1)

array([22989.7102504 , 22290.8873145 , 29759.45914857, ...,
       11784.57813127, 53851.37538931, 19117.72744361])